In [1]:
import pandas as pd
import os
os.getcwd()
import warnings
warnings.filterwarnings(action="ignore")

%matplotlib inline 
import matplotlib as mpl 
import matplotlib.pyplot as plt 
# To plot pretty figures 
mpl.rc('axes', labelsize=14) 
mpl.rc('xtick', labelsize=12) 
mpl.rc('ytick', labelsize=12) 

plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签 
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号 

'C:\\Users\\L_98B'

In [2]:
inputfile = "taobao.csv"
outputfile = "anchor_tb.txt"
data = pd.read_csv(inputfile, encoding = 'utf-8')
data.head()

,anchorUserID,username,type,orderTime,content,anchor,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,蛋蛋***逗逗,直播,2020.07.20,超好得直播,薇娅viya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,五和***餐具,直播,2020.07.16,非常好,薇娅viya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,宝拉***ip,直播,2020.07.13,薇娅姐的带货力没的说，团队很奈斯~~~期待下一次~~~,薇娅viya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,今生***仙花,直播,2020.07.09,非常满意,薇娅viya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,le***an,直播,2020.07.08,NaN,薇娅viya,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data = data[[u'content']]
data.to_csv(outputfile, index = False, header = False, encoding = 'utf-8')
data.head()

,content
0,超好得直播
1,非常好
2,薇娅姐的带货力没的说，团队很奈斯~~~期待下一次~~~
3,非常满意
4,NaN


In [45]:
# 文本评论去重
data = pd.read_csv(outputfile, encoding = 'utf-8', header = None)
outdata = data
data.head()

data = pd.DataFrame(data[0].unique())
data.to_csv(outputfile, index = False, header = False, encoding = 'utf-8')
print(u"共有%s 条有效评论"%(len(data)))

共有701 条有效评论


In [46]:
# 分析系统默认评价是否存在
series_data = pd.Series(outdata[0])
fre_data = pd.DataFrame(series_data.value_counts())
fre_data.head(20)

,0
很好很负责，期待长期合作,58
很好,42
好评,38
非常满意,33
很厉害，出货主播，牛！,32
大热天的，停电了在楼道坚持直播1个小时，佩服，不成功都难,32
ok,32
给力,32
一如既往的好，完美！,32
lihai,32


In [64]:
# 文本评论分词
import jieba
data = pd.read_csv(outputfile, encoding='utf-8', header=None).astype(str)
mycut = lambda s: ' '.join(jieba.cut(s))
data = data[0].apply(mycut)
data

0                                                超好 得 直播
1                                                   非常 好
2       薇 娅 姐 的 带 货力 没 的 说 ， 团队 很 奈斯 ~ ~ ~ 期待 下 一次 ~ ~ ~
3                                                  非常 满意
4                                                    nan
                             ...                        
696    真的 差 ， 太差 了 ， 显示 30 万 粉丝 ， 为什么 到 店 流量 才 几百 ， 我...
697    不想 多 说 ， 做 了 3 天 ， 如果 自己 不去 问 自己 的 产品 ， 根本 不会 ...
698    真的 是 太差 ， 一场 下来 才 卖 2 单 ， 转化率 非常低 。 最 气愤 的 是 让...
699    差评 ， 差评 ， 差评 ， 就是 一个 骗钱 的 骗子 ， 主播 收钱 之后 ， 并 没有...
700    大家 不要 上当 了 ， 这家 根本就是 骗钱 的 ， 粉丝 虽然 多 但是 没有 什么 转...
Name: 0, Length: 701, dtype: object

In [98]:
# 情感分析
from snownlp import SnowNLP 
pos = []
neg = []
outputfile1 = 'pos.txt'
outputfile2 = 'neg.txt'
for i in data:
    if(SnowNLP(i).sentiments < 0.2):
        neg.append(i)
    else:
        pos.append(i)

pos = pd.DataFrame(pos)
neg = pd.DataFrame(neg)
pos.to_csv(outputfile1, index = False, header = False, encoding = 'utf-8') 
neg.to_csv(outputfile2, index = False, header = False, encoding = 'utf-8')

In [101]:
# 去掉停用词
import os

stoplist = 'stoplist.txt'

stop = pd.read_csv(stoplist, encoding='utf-8', header=None, sep = 'tipdm')
stop = [' ', ''] + list(stop[0]) + list(stop[0])

neg[1] = neg[0].apply(lambda s: s.split(' '))
neg[2] = neg[1].apply(lambda x: [i for i in x if i not in stop])
pos[1] = pos[0].apply(lambda s: s.split(' '))
pos[2] = pos[1].apply(lambda x: [i for i in x if i not in stop])

print(neg.head())
print(pos.head())



D:\Anaconda\lib\site-packages\ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


                                                   0  \
0                                   支持 薇 娅 ， 从未 失望 过   
1                一姐 就是 一姐 ， 真的 超级 腻害 拉 ， 长期 合作 长期 合作   
2                              很棒 ， 已 爆单 ， 下次 还 合作 ~   
3                           简直 是 太 爆炸 了 ， 女神 ， 厉害 厉害   
4  一姐 就 一姐 ， 爆 了 3 万件 ， 是 我 货 不够 卖 ， 要不然 至少 再 多 2 万件   

                                                   1  \
0                           [支持, 薇, 娅, ，, 从未, 失望, 过]   
1  [一姐, 就是, 一姐, ，, 真的, 超级, 腻害, 拉, ，, 长期, 合作, 长期, 合作]   
2                    [很棒, ，, 已, 爆单, ，, 下次, 还, 合作, ~]   
3                [简直, 是, 太, 爆炸, 了, ，, 女神, ，, 厉害, 厉害]   
4  [一姐, 就, 一姐, ，, 爆, 了, 3, 万件, ，, 是, 我, 货, 不够, 卖,...   

                                         2  
0                           [支持, 薇, 娅, 失望]  
1  [一姐, 一姐, 真的, 超级, 腻害, 拉, 长期, 合作, 长期, 合作]  
2                      [很棒, 已, 爆单, 下次, 合作]  
3                      [太, 爆炸, 女神, 厉害, 厉害]  
4     [一姐, 一姐, 爆, 万件, 货, 不够, 卖, 至少, 再, 万件]  
                                                  0  \
0

In [100]:
# LDA 模型主题分析

# 修改默认编码 
import sys 
sys.getdefaultencoding()  # 查看当前编码格式 
# 报错时（AttributeError: 'module' object has no attribute 'setdefaultencoding'） 
# reload(sys)  # 报上面的错时，执行此命令 
# sys.setdefaultencoding('utf8')  # 修改编码格式

'utf-8'

In [102]:
from gensim import corpora, models 
#负面主题分析 
neg_dict = corpora.Dictionary(neg[2]) #建立词典 
neg_corpus = [neg_dict.doc2bow(i) for i in neg[2]] #建立语料库 
neg_lda = models.LdaModel(neg_corpus, num_topics = 3, id2word = neg_dict) #LDA模型训练 
for i in range(3):  
    print(neg_lda.show_topic(i))#展示主题

[('OK', 0.06778369), ('主播', 0.020990489), ('合作', 0.01336681), ('粉丝', 0.009211372), ('对接', 0.008446066), ('效果', 0.0076208836), ('好', 0.0069110706), ('介绍', 0.006315871), ('差评', 0.0060789865), ('不想', 0.0057546864)]
[('主播', 0.019570278), ('OK', 0.01811344), ('直播', 0.017269108), ('回复', 0.014519138), ('一个', 0.010976655), ('合作', 0.010444205), ('钱', 0.009467464), ('卖', 0.008453696), ('介绍', 0.008395308), ('之后', 0.007998551)]
[('合作', 0.039410897), ('主播', 0.02169453), ('直播', 0.013145172), ('骗子', 0.012187796), ('真的', 0.011931434), ('长期', 0.010626094), ('粉丝', 0.009163836), ('找', 0.0077829), ('商家', 0.0075435624), ('好', 0.0070889913)]


In [103]:
#正面主题分析 
pos_dict = corpora.Dictionary(pos[2]) 
pos_corpus = [pos_dict.doc2bow(i) for i in pos[2]] 
pos_lda = models.LdaModel(pos_corpus, num_topics = 3, id2word = pos_dict) 
for i in range(3):  
    print(pos_lda.show_topic(i))#展示主题 

[('赞赞', 0.061647836), ('合作', 0.032308213), ('主播', 0.024865624), ('好', 0.023437804), ('带货', 0.018372351), ('娅', 0.017319014), ('薇', 0.0155770825), ('超级', 0.014707273), ('效果', 0.013842887), ('能力', 0.012337648)]
[('好', 0.075421385), ('薇', 0.046534255), ('娅', 0.045201883), ('合作', 0.041676372), ('效果', 0.024063261), ('棒', 0.020251168), ('主播', 0.019633647), ('棒棒', 0.016801884), ('期待', 0.016509831), ('团队', 0.016110959)]
[('主播', 0.059830647), ('合作', 0.050328508), ('好', 0.04331129), ('效果', 0.023883278), ('不错', 0.020253625), ('期待', 0.017515074), ('小茶', 0.012156421), ('直播', 0.012094463), ('负责', 0.01205182), ('满意', 0.011719543)]


In [104]:
# 可视化
import pyLDAvis.gensim 
pyLDAvis.enable_notebook()  #设置显示方式 

pyLDAvis.gensim.prepare(neg_lda, neg_corpus, neg_dict) 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.071861  0.004091       1        1  39.541332
1     -0.030336 -0.041448       2        1  35.843876
0     -0.041526  0.037357       3        1  24.614790, topic_info=    Term       Freq      Total Category  logprob  loglift
88    OK  28.000000  28.000000  Default  30.0000  30.0000
5     合作  27.000000  27.000000  Default  29.0000  29.0000
162   回复   8.000000   8.000000  Default  28.0000  28.0000
314   对接   6.000000   6.000000  Default  27.0000  27.0000
66     好   5.000000   5.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
228    找   1.221776   6.534324   Topic3  -5.4921  -0.2749
191    钱   1.194173   8.086832   Topic3  -5.5150  -0.5110
205    下   1.062419   4.102254   Topic3  -5.6319   0.0508
43    直播   1.145256  14.867427   Topic3  -5.5568  -1.1617
162   回复   1.041001   8.329750   Topic3  -5.6522  -0.6778

[200 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
589       1  0.515013  1800
409       1  0.362455   500
409       2  0.724909   500
204       2  0.715496   680
88        2  0.283805    OK
...     ...       ...   ...
304       1  0.496287    骗钱
304       2  0.496287    骗钱
304       3  0.248143    骗钱
338       1  0.612229     黑
338       2  0.306115     黑

[224 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [105]:
pos_theme = pos_lda.show_topics() #展示主题
print(pos_theme)
neg_theme = neg_lda.show_topics() #展示主题
print(neg_theme)

[(0, '0.062*"赞赞" + 0.032*"合作" + 0.025*"主播" + 0.023*"好" + 0.018*"带货" + 0.017*"娅" + 0.016*"薇" + 0.015*"超级" + 0.014*"效果" + 0.012*"能力"'), (1, '0.075*"好" + 0.047*"薇" + 0.045*"娅" + 0.042*"合作" + 0.024*"效果" + 0.020*"棒" + 0.020*"主播" + 0.017*"棒棒" + 0.017*"期待" + 0.016*"团队"'), (2, '0.060*"主播" + 0.050*"合作" + 0.043*"好" + 0.024*"效果" + 0.020*"不错" + 0.018*"期待" + 0.012*"小茶" + 0.012*"直播" + 0.012*"负责" + 0.012*"满意"')]
[(0, '0.068*"OK" + 0.021*"主播" + 0.013*"合作" + 0.009*"粉丝" + 0.008*"对接" + 0.008*"效果" + 0.007*"好" + 0.006*"介绍" + 0.006*"差评" + 0.006*"不想"'), (1, '0.020*"主播" + 0.018*"OK" + 0.017*"直播" + 0.015*"回复" + 0.011*"一个" + 0.010*"合作" + 0.009*"钱" + 0.008*"卖" + 0.008*"介绍" + 0.008*"之后"'), (2, '0.039*"合作" + 0.022*"主播" + 0.013*"直播" + 0.012*"骗子" + 0.012*"真的" + 0.011*"长期" + 0.009*"粉丝" + 0.008*"找" + 0.008*"商家" + 0.007*"好"')]


In [106]:
# 提取高频词
import re
pattern = re.compile(r'[\u4e00-\u9fa5]+')
# 主题特征词
n1 = pattern.findall(pos_theme[0][1])
n2 = pattern.findall(pos_theme[1][1])
n3 = pattern.findall(pos_theme[2][1])
print(n1)
print(n2)
print(n3)
m1 = pattern.findall(neg_theme[0][1])
m2 = pattern.findall(neg_theme[1][1])
m3 = pattern.findall(neg_theme[2][1])
print(m1)
print(m2)
print(m3)

['赞赞', '合作', '主播', '好', '带货', '娅', '薇', '超级', '效果', '能力']
['好', '薇', '娅', '合作', '效果', '棒', '主播', '棒棒', '期待', '团队']
['主播', '合作', '好', '效果', '不错', '期待', '小茶', '直播', '负责', '满意']
['主播', '合作', '粉丝', '对接', '效果', '好', '介绍', '差评', '不想']
['主播', '直播', '回复', '一个', '合作', '钱', '卖', '介绍', '之后']
['合作', '主播', '直播', '骗子', '真的', '长期', '粉丝', '找', '商家', '好']


In [107]:
# 取得每个主题的特征词 
# 取得每个主题的特征词 
pos_key_words=[] 
for i in range(3): 
    pos_key_words.append(pattern.findall(pos_theme[i][1]))

# 变成 DataFrame 格式 
pos_key_words = pd.DataFrame(data=pos_key_words,index=['主题1',"主题2","主题3"]) 
pos_key_words
for i in range(3): 
    pos_key_words.append(pattern.findall(pos_theme[i][1]))

# 变成 DataFrame 格式 
pos_key_words = pd.DataFrame(data=pos_key_words,index=['主题1',"主题2","主题3"]) 
pos_key_words

,0,1,2,3,4,5,6,7,8,9
主题1,赞赞,合作,主播,好,带货,娅,薇,超级,效果,能力
主题2,好,薇,娅,合作,效果,棒,主播,棒棒,期待,团队
主题3,主播,合作,好,效果,不错,期待,小茶,直播,负责,满意


In [109]:
# 取得每个主题的特征词 
neg_key_words=[] 
for i in range(3): 
    neg_key_words.append(pattern.findall(neg_theme[i][1]))

# 变成 DataFrame 格式 
neg_key_words = pd.DataFrame(data=neg_key_words,index=['主题1',"主题2","主题3"]) 
neg_key_words

,0,1,2,3,4,5,6,7,8,9
主题1,主播,合作,粉丝,对接,效果,好,介绍,差评,不想,None
主题2,主播,直播,回复,一个,合作,钱,卖,介绍,之后,None
主题3,合作,主播,直播,骗子,真的,长期,粉丝,找,商家,好
